# Query Studio: Overview of Text-to-SQL Pipeline

In the era of data-driven decision-making, the ability to query databases efficiently and accurately is important. However, not everyone is proficient in SQL. This is where Text-to-SQL comes in. Text-to-SQL systems enable users to interact with databases using natural language, making data querying accessible to a broader audience. 

In this notebook, we will explore how Text-to-SQL pipeline is designed to work and how we can somewhat optimize the conversion of langauge questions into SQL queries using Large Language Models. By the end of the article, we will have a good understanding of how Text-to-SQL works and how they can transform the way we interact with data. 



In [1]:
import os
from dotenv import load_dotenv

if os.getcwd().endswith("notebooks"):
    os.chdir("./../../query_studio")
    print("Current Working Directory:", os.getcwd())

load_dotenv()

ANTHROPIC_KEY = os.getenv("ANTHROPIC_API_KEY")
ANTHROPIC_MODEL = os.getenv("ANTHROPIC_MODEL")
print("Anthropic model to be used:", ANTHROPIC_MODEL)

Current Working Directory: /Users/rafaelmadrigal/Local Documents/Code-Work/otacta/query-studio/query_studio
Anthropic model to be used: claude-3-5-sonnet-20241022


In [2]:
from langchain_anthropic import ChatAnthropic

llm = ChatAnthropic(model=ANTHROPIC_MODEL, api_key=ANTHROPIC_KEY, temperature=0.5)
llm.invoke("hi")

AIMessage(content='Hello! How can I help you today?', additional_kwargs={}, response_metadata={'id': 'msg_01Rdkww3o38bPdbao9ZsXpzN', 'model': 'claude-3-5-sonnet-20241022', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 8, 'output_tokens': 12}}, id='run-16271d1c-e7f3-4415-b0ee-e702727ec5ca-0', usage_metadata={'input_tokens': 8, 'output_tokens': 12, 'total_tokens': 20, 'input_token_details': {'cache_read': 0, 'cache_creation': 0}})

## Understanding QueryGenerator

The `QueryGenerator` class is responsible for generating synthetic natural language questions based on a given `table_schema` and `table_description` and optimizing these questions for SQL conversion. Let's take a look at how it works. 

### **Initialization and Setup**

The `QueryGenerator` class is initialized with a language model (`llm`), `model name`, and API key (`api_key`). If the language model is not provided, it loads the environment variables to get the model name and API key. The initialization process also involves building the chains for generating and optimizing questions.

### **Fitting the QueryGenerator**
The `fit()` method configures the QueryGenerator with table descriptions and schema. If these are not provided, it uses default values.

```python
def fit(self, table_descriptions: str = None, table_schema: str = None) -> None:
    if (table_descriptions is None) or (table_schema is None):
        warnings.warn("No table descriptions or schema provided. Using default values.")

    self.table_descriptions = table_descriptions if table_descriptions else DEFAULT_TABLE_DESCRIPTIONS
    self.table_schema = table_schema if table_schema else DEFAULT_TABLE_SCHEMA
```



In [3]:
from studio.query_generator import QueryGenerator
from studio.text_to_sql import Text2SQLAgent

In [4]:
TABLE_DESCRIPTIONS = """
Table Purposes and Context:

1. Fulfillment Table:
    - Purpose: Captures qualitative data for market research
    - Contains customer instructions and notes
    - Good for analyzing customer preferences and special requests

2. Orders Table:
    - Central hub for managing and recording customer orders
    - Tracks complete order lifecycle from placement to fulfillment
    - Enables descriptive analytics of transactions
    - Supports quantitative market research
    - Primary source for transaction analysis

3. Daily_Sales Table:
    - Provides daily business performance summaries
    - Consolidates data for operational snapshots
    - Supports forecasting and trend analysis
    - Enables quick report generation with reduced lag time
    - Lighter alternative to detailed transaction reports

4. Orders_Itemized Table:
    - Tracks individual line items within orders
    - Provides granular transaction details
    - Supports detailed product-level analysis
    - Ensures financial compliance and transparency
    - Enables precise reporting on product sales

5. Product_Sales Table:
    - Focuses on product-specific sales tracking
    - Records variants, quantities, and pricing
    - Supports product performance analytics
    - Enables trend analysis at product level
    - Tracks daily product revenue
"""

TABLE_SCHEMA = """
Table: fulfillment
- item_instructions (text): Customer-provided notes for specific items
- order_notes (text): General order-related customer notes
- fulfillment_date (timestamp): When customer instructions were provided

Table: orders
- order_id (varchar): Primary key, unique order identifier
- order_time (timestamp): When order was placed
- subtotal (decimal): Pre-tax/fees order amount
- taxable_items (decimal): Amount for tax-eligible items
- non_taxable_items (decimal): Amount for tax-exempt items (bulk purchases)
- delivery_fee (decimal): Standard delivery charge
- tax (decimal): Total tax applied
- stripe_tendered (decimal, nullable): Online payment amount via Stripe
- total_payments_tendered (decimal): Total paid across all methods
- gift_cards_purchased (decimal): New gift card purchase value
- gift_cards_tendered (decimal): Payment via existing gift cards
- refunded (decimal): Total refund amount
- net_sales (decimal): Final amount after all adjustments
- items (text): Detailed item list with quantities
- fulfillment_method (varchar): PICKUP, DELIVERY, or Point of Sale
- fulfillment_time (timestamp): Pickup/delivery completion time
- city (varchar): Customer city (online orders)
- province (varchar): Customer province (online orders)
- postal_code (varchar): Customer postal code (online orders)

Table: daily_sales
- date (date): Primary key for daily summary
- orders_count (integer): Total daily orders
- delivery_fees_count (integer): Number of deliveries
- delivery_fees_amount (decimal): Total standard delivery fees
- priority_express_delivery (decimal): Total expedited delivery fees
- tax (decimal): Total daily tax collected
- stripe_tendered_online (decimal): Total online payments
- gift_cards_purchased (decimal): New gift card sales
- gift_cards_tendered (decimal): Gift card payments
- refunded (decimal): Total refunds
- net_sales (decimal): Final daily revenue

Table: orders_itemized
- order_id (varchar): References orders.order_id
- time_placed (timestamp): Item addition time
- item_category (varchar): Product, Add-on, Tax, Tips, Fulfillment
- item_name (varchar): Item description
- product_type (varchar): Product category
- product_variant (varchar, nullable): Product SKU
- quantity (integer): Number of units
- unit_price (decimal): Base price per unit
- subtotal (decimal): Pre-tax/fees amount
- delivery_fee (decimal): Item delivery charge
- tax (decimal): Item tax amount
- total (decimal): Total with tax and fees
- gift_card_purchased (decimal): Gift card value if applicable
- net_sales (decimal): Final item revenue
- fulfillment_method (varchar): Delivery or Pickup

Table: product_sales
- product (varchar): Product name
- variant (varchar): Product SKU
- sale_date (date): Sale date
- count (integer): Units sold
- unit_price (decimal): Price per unit
- subtotal (decimal): Total revenue
- tax (decimal): Tax collected

Key Relationships and Notes:
1. orders_itemized.order_id links to orders.order_id for detailed transaction lookup
2. daily_sales aggregates orders data by date for summary reporting
3. product_sales aggregates orders_itemized by product and date
4. fulfillment links to orders for customer instructions
5. Online orders (stripe_tendered not null) include customer location data
6. Bulk purchases (1/2 or 1 Dozen) are tax-exempt
"""

In [5]:
generator = QueryGenerator(llm=llm)

generator.fit(table_descriptions=TABLE_DESCRIPTIONS, table_schema=TABLE_SCHEMA)


## **Generating Natural Language Questions**

#### Looking at `generate_nl_question`
The `generate_nl_questions` method generates a specific number of natural language questions based on the table descriptions and schema. It uses the generator chain and a retry mechanism to ensure successful generation

```python
async def generate_nl_questions(self, n: str) -> t.List[Question]:
    input_dict = dict(
        table_description=self.table_descriptions,
        table_schema=self.table_schema, 
        n_questions=n
    )
    
    results = await self._generate_with_retry(
        self.generator_chain, 
        input_dict, 
        self.max_retries
    )
    
    results = json.loads(results)

    return [Question(question=q.pop("question"), metadata=q) for q in results.get("questions")]
```

#### Looking at `NL_QUESTION_GENERATOR_PROMPT`

The `NL_QUESTION_GENERATOR_PROMPT` is designed to guide the language model in generating authentic business questions focused solely on descriptive analytics. These questions should reflect real business needs for understanding historical performance and current states, not predicitons or prescriptions

The prompt provides detailed role contexts for both business owners and analysts. This helps the language model understand the perspectives and needs of these roles, ensuring that the generated questions are relevant and useful

Role Context: (in the prompt)

1. Business Owner Perspective:
   - Wants to understand what has happened in the business
   - Needs summaries of performance metrics
   - Interested in key trends and patterns in historical data
   - Seeks to understand current business state

2. Analyst Perspective:
   - Focuses on data aggregation and summarization
   - Creates reports showing what has occurred
   - Identifies notable patterns in historical data
   - Organizes and categorizes past performance


In [6]:
nl_questions = await generator.generate_nl_questions(n=10)
nl_questions

[Question(question='How has our gift card usage trended over the past year compared to new gift card purchases?', metadata={'business_context': 'Understanding the balance between gift card sales and redemptions to gauge program effectiveness', 'role': 'Owner'}),
 Question(question="What's the current split between tax-exempt bulk orders and regular taxable purchases across our different fulfillment methods?", metadata={'business_context': 'Analyzing distribution of bulk vs. regular orders to understand customer purchasing patterns', 'role': 'Analyst'}),
 Question(question='Can you show me how our delivery fees revenue has changed month-over-month since we introduced priority express delivery?', metadata={'business_context': 'Evaluating the revenue impact of different delivery service options', 'role': 'Owner'}),
 Question(question="What's been the typical breakdown of special instructions in our online orders by product category?", metadata={'business_context': 'Understanding customer 

## Optimizing Questions for SQL conversion
The `optimize_query` method refines the generated natural language questions to make them more suitable for SQL conversion. 

**Here are some key reasons why this optimization step is important**

1. Improving Clarity and Precisions
    - Natural Language questions can often be ambiguous or imprecise. The optimization process helps to clarify the intent of the question and ensures that it is specific enough to be accurately converted into an SQL query. This reduces the chances of generating incorrect or inefficient SQL code

2. Maintaining Context and Relevance
    - The optimization process takes into account table descriptions and schema ensuring that the questions remain relevnat to the database structure. This helps maintain the context and ensures that the generated SQL queries are meaningful and aligned with available data

**Example**

Natural Language Question: "Show me the total sales for each month this year."
Without Optimization, the resulting SQL query might look like

```sql
    SELECT 
        month, 
        SUM(sales) 
    FROM 
        orders 
    WHERE 
        order_time >= '2025-01-01' 
    GROUP BY 
        month;
```

Potential Issues:
- the column `month` might not exist directly in the orders table
- the date filter might not be correctly aligned with the current year
- the table name might not include the correct schema prefix

With Optimization, the natural language query can be translated to: "Calculate the sum of net_sales from daily_sales table, grouped by extract(month from date) where date >= '2024-01-01' and date <= '2024-12-31'". Resulting to a better SQL generated code

```sql
    SELECT 
        EXTRACT(MONTH FROM order_time) AS month,
        SUM(total_sales) AS total_sales
    FROM 
        gold.orders 
    WHERE 
        order_time >= DATE_TRUNC('year', CURRENT_DATE)
    GROUP BY 
        EXTRACT(MONTH FROM order_time)
    ORDER BY 
        month;
```

Key Improvements include:
- The EXTRACT(MONTH FROM order_time) function ensures that the month is correctly derived from the order_time column.
- The DATE_TRUNC('year', CURRENT_DATE) function ensures that the date filter is aligned with the current year.
- The table name includes the correct schema prefix (gold.orders), ensuring that the query is executed against the correct table.

#### Looking at `optimize_query` 

When `optimize_query` is called, it iterates through the list of questions, constructs an input dictionary for each question and passes it to the optimizer chain. The prompt `QUERY_OPTIMIZATION_PROMPT` guides the language model to produce an optimized version of the question


```python
async def optimize_query(self, questions: t.Union[t.List[str], t.List[Question]]) -> t.List[Question]:
    revised_questions = []
    
    for question in tqdm(questions):
        if isinstance(question, Question):
            q = question.question
            metadata = question.metadata
        elif isinstance(question, str):
            q = question
            metadata = {}
        
        input_dict = dict(
            table_descriptions=self.table_descriptions,
            table_schema=self.table_schema, 
            question=q
        )

        results = await self._generate_with_retry(
            self.optimizer_chain,
            input_dict,
            self.max_retries
        )   
        
        results = json.loads(results)
        
        revised_questions.append(
            Question(
                question=results.pop("optimized_question"),
                metadata={**results, "nl_question": q, **metadata}
            )
        )

    return revised_questions
```

#### Looking at  `QUERY_OPTIMIZATION_PROMPT`

The prompt provides specific optimization instructions on how to optimize the question for SQL. These instructions include:
- Using exact table and column names from the schema
- Making implicit joins explicit
- Clarifying aggregation functions
- Specifying grouping criteria
- Making filters explicit
- Preserving time ranges mentioned in the original question
- Using appropriate table based on the analysis needed

If the original question is vague or unclear, the prompt instructs the language model to select the most reasonable interpretation based on the business context.

The prompt also provibes examples of original and optimized questions to illustrate how the optimization should be done. 

```python
Original: "How are our online sales doing?"
{{
    "optimized_question": "Calculate the sum of net_sales from orders table where stripe_tendered is not null, grouped by date"
}}

Original: "Which products sell best in each city?"
{{
    "optimized_question": "Find the products with highest count and sum of net_sales from orders_itemized joined with orders, grouped by product_type and city"
}}

Original: "What's our gift card usage like?"
{{
    "optimized_question": "Calculate the sum of gift_cards_purchased and gift_cards_tendered from orders table, grouped by order_time by month"
}}
```





In [7]:
from studio.models import Question

nl_questions = [
    Question(
        question="What is the total daily sales revenue over the past month?",
        metadata={},
    ),
    Question(
        question="Which cookie variant had the highest sales in the past two weeks?",
        metadata={},
    ),
    Question(question="Which cookies tend to sell out the fastest?", metadata={}),
]

In [8]:
sql_questions = await generator.optimize_query(nl_questions)
sql_questions

100%|██████████| 3/3 [00:08<00:00,  2.73s/it]


[Question(question="Calculate the sum of net_sales from daily_sales table where date >= CURRENT_DATE - INTERVAL '1 month', grouped by date order by date", metadata={'nl_question': 'What is the total daily sales revenue over the past month?'}),
 Question(question='Select the product_variant from product_sales table with the highest sum of count where sale_date is within the last 14 days', metadata={'nl_question': 'Which cookie variant had the highest sales in the past two weeks?'}),
 Question(question="Calculate the average time between orders for each product_variant in orders_itemized where product_type = 'Cookie', ordered by the shortest average time interval", metadata={'nl_question': 'Which cookies tend to sell out the fastest?'})]

## Understanding Text2SQLAgent Class

The Text2SQLAgent class is designed to convert natural language questions into SQL queries and execute them against a database. This class leverages advanced language models and integrates with SQL databases to provide accurate and efficient query results.

When we initialize the class, the method `_get_sql_agent_executor` is called. It configures the agent executor with the language model, SQL Database toolkit, and additional settings such as handling parsing errors and returning intermediate steps. 

### Generating SQL from Text from `generate_sql_from_text` method
- This method generates SQL code and a chain of thought for a given list of questions
- It iterates through the questions, generate the SQL code and chain of thought for each question using the `_generate_sql_and_chain_of_thought` method and returns the results

In [9]:

DEFAULT_TABLE_COLUMNS = {
    "gold.fulfillment": ["item_instructions", "order_notes", "fulfillment_date"],
    "gold.orders": [
        "order_id",
        "order_time",
        "subtotal",
        "taxable_items",
        "non_taxable_items",
        "delivery_fee",
        "tax",
        "stripe_tendered",
        "total_payments_tendered",
        "gift_cards_purchased",
        "gift_cards_tendered",
        "refunded",
        "net_sales",
        "items",
        "fulfillment_method",
        "fulfillment_time",
        "city",
        "province",
        "postal_code",
    ],
    "gold.daily_sales": [
        "date",
        "orders_count",
        "delivery_fees_count",
        "delivery_fees_amount",
        "priority_express_delivery",
        "tax",
        "stripe_tendered_online",
        "gift_cards_purchased",
        "gift_cards_tendered",
        "refunded",
        "net_sales",
    ],
    "gold.orders_itemized": [
        "order_id",
        "time_placed",
        "item_category",
        "item_name",
        "product_type",
        "product_variant",
        "quantity",
        "unit_price",
        "subtotal",
        "delivery_fee",
        "tax",
        "total",
        "gift_card_purchased",
        "net_sales",
        "fulfillment_method",
    ],
}

In [10]:
agent = Text2SQLAgent(llm, table_columns=DEFAULT_TABLE_COLUMNS)
results = await agent.generate_sql_from_text(sql_questions)

  0%|          | 0/3 [00:00<?, ?it/s]



> Entering new SQL Agent Executor chain...
Let me help you generate the SQL query for calculating the sum of net_sales from the daily_sales table for the last month.

Action: sql_db_query_checker
Action Input: 
```sql
SELECT 
    date,
    SUM(net_sales) as total_net_sales
FROM gold.daily_sales
WHERE date >= CURRENT_DATE - INTERVAL '1 month'
GROUP BY date
ORDER BY date;
``````sql
SELECT 
    date,
    SUM(net_sales) as total_net_sales
FROM gold.daily_sales
WHERE date >= CURRENT_DATE - INTERVAL '1 month'
GROUP BY date
ORDER BY date;
```Let me execute the query to get the results.

Action: sql_db_query
Action Input: 
```sql
SELECT 
    date,
    SUM(net_sales) as total_net_sales
FROM gold.daily_sales
WHERE date >= CURRENT_DATE - INTERVAL '1 month'
GROUP BY date
ORDER BY date;
```Error: (psycopg2.errors.SyntaxError) syntax error at or near "```"
LINE 1: ```sql
        ^

[SQL: ```sql
SELECT 
    date,
    SUM(net_sales) as total_net_sales
FROM gold.daily_sales
WHERE date >= CURRENT_DATE

 33%|███▎      | 1/3 [00:26<00:53, 26.72s/it]



> Entering new SQL Agent Executor chain...
I notice that the user's query mentions a "product_sales" table which is not present in the provided schema. The available tables are in the 'gold' schema and include: fulfillment, orders, daily_sales, and orders_itemized.

Based on the user's intent to find product variants with highest sales in the last 14 days, we can use the orders_itemized table which contains product_variant and sales information.

Let me write a query to get this information:

Action: sql_db_query_checker
Action Input: 
SELECT 
    product_variant,
    COUNT(*) as sale_count,
    SUM(quantity) as total_quantity
FROM gold.orders_itemized
WHERE time_placed >= CURRENT_DATE - INTERVAL '14 days'
GROUP BY product_variant
ORDER BY total_quantity DESC
LIMIT 10;
SELECT 
    product_variant,
    COUNT(*) as sale_count,
    SUM(quantity) as total_quantity
FROM gold.orders_itemized
WHERE time_placed >= CURRENT_DATE - INTERVAL '14 days'
GROUP BY product_variant
ORDER BY total_quan

 67%|██████▋   | 2/3 [00:44<00:21, 21.27s/it]



> Entering new SQL Agent Executor chain...
Let me help you create a SQL query for this analysis.

Action: sql_db_query_checker
Action Input: WITH time_differences AS (
  SELECT 
    product_variant,
    time_placed,
    LAG(time_placed) OVER (PARTITION BY product_variant ORDER BY time_placed) as prev_order_time,
    EXTRACT(EPOCH FROM (time_placed - LAG(time_placed) OVER (PARTITION BY product_variant ORDER BY time_placed)))/3600 as hours_between_orders
  FROM gold.orders_itemized
  WHERE product_type = 'Cookie'
)
SELECT 
  product_variant,
  AVG(hours_between_orders) as avg_hours_between_orders,
  COUNT(*) as total_orders
FROM time_differences
WHERE hours_between_orders IS NOT NULL
GROUP BY product_variant
ORDER BY avg_hours_between_orders ASC
LIMIT 10;
WITH time_differences AS (
  SELECT 
    product_variant,
    time_placed,
    LAG(time_placed) OVER (PARTITION BY product_variant ORDER BY time_placed) as prev_order_time,
    EXTRACT(EPOCH FROM (time_placed - LAG(time_placed) OVER (

100%|██████████| 3/3 [01:47<00:00, 35.92s/it]



## Showing the Results

After generating and optimizing the natural language questions, we proceed to convert these questions into SQL queries using the `Text2SQLAgent` class. The results of these conversions are stored in the `results` variable. Each entry in the `results` list contains the original question, the generated SQL code, the chain of thought, and the final output.

Here are the results:

1. **Total Daily Sales Revenue Over the Past Month**
    - **Original Question:** What is the total daily sales revenue over the past month?
    - **Generated SQL:**
      ```sql
      SELECT date, SUM(net_sales) as total_net_sales
      FROM gold.daily_sales
      WHERE date >= CURRENT_DATE - INTERVAL '1 month'
      GROUP BY date
      ORDER BY date;
      ```
    - **Output:** The query to get the daily net sales for the past month is:
      ```sql
      SELECT date, SUM(net_sales) as total_net_sales
      FROM gold.daily_sales
      WHERE date >= CURRENT_DATE - INTERVAL '1 month'
      GROUP BY date
      ORDER BY date;
      ```

2. **Highest Sales in the Past Two Weeks**
    - **Original Question:** Which cookie variant had the highest sales in the past two weeks?
    - **Generated SQL:**
      ```sql
      SELECT 
            product_variant,
            SUM(quantity) as total_quantity
      FROM gold.orders_itemized
      WHERE order_time >= CURRENT_DATE - INTERVAL '14 days'
      GROUP BY product_variant
      ORDER BY total_quantity DESC
      LIMIT 10;
      ```
    - **Output:** Based on the sales data from the last 14 days, here are the top product variants by quantity sold.

3. **Cookies That Tend to Sell Out the Fastest**
    - **Original Question:** Which cookies tend to sell out the fastest?
    - **Generated SQL:**
      ```sql
      SELECT 
            product_type,
            product_variant,
            AVG(quantity) as avg_daily_sales_count
      FROM gold.orders_itemized
      WHERE product_type = 'cookie'
      GROUP BY product_type, product_variant
      ORDER BY avg_daily_sales_count DESC
      LIMIT 10;
      ```
    - **Output:** Here are the top 10 cookie products by average daily sales quantity.


In [11]:
results

[{'input': "Calculate the sum of net_sales from daily_sales table where date >= CURRENT_DATE - INTERVAL '1 month', grouped by date order by date",
  'sql_code': "\nSELECT \n    date,\n    SUM(net_sales) as total_net_sales\nFROM gold.daily_sales\nWHERE date >= CURRENT_DATE - INTERVAL '1 month'\nGROUP BY date\nORDER BY date;\n",
  'chain_of_thought': [{'action': 'sql_generation',
    'input': "Generate SQL code for the following user query using the schema `gold` and the defined columns: {'gold.fulfillment': ['item_instructions', 'order_notes', 'fulfillment_date'], 'gold.orders': ['order_id', 'order_time', 'subtotal', 'taxable_items', 'non_taxable_items', 'delivery_fee', 'tax', 'stripe_tendered', 'total_payments_tendered', 'gift_cards_purchased', 'gift_cards_tendered', 'refunded', 'net_sales', 'items', 'fulfillment_method', 'fulfillment_time', 'city', 'province', 'postal_code'], 'gold.daily_sales': ['date', 'orders_count', 'delivery_fees_count', 'delivery_fees_amount', 'priority_express_

In [12]:
results[0]

{'input': "Calculate the sum of net_sales from daily_sales table where date >= CURRENT_DATE - INTERVAL '1 month', grouped by date order by date",
 'sql_code': "\nSELECT \n    date,\n    SUM(net_sales) as total_net_sales\nFROM gold.daily_sales\nWHERE date >= CURRENT_DATE - INTERVAL '1 month'\nGROUP BY date\nORDER BY date;\n",
 'chain_of_thought': [{'action': 'sql_generation',
   'input': "Generate SQL code for the following user query using the schema `gold` and the defined columns: {'gold.fulfillment': ['item_instructions', 'order_notes', 'fulfillment_date'], 'gold.orders': ['order_id', 'order_time', 'subtotal', 'taxable_items', 'non_taxable_items', 'delivery_fee', 'tax', 'stripe_tendered', 'total_payments_tendered', 'gift_cards_purchased', 'gift_cards_tendered', 'refunded', 'net_sales', 'items', 'fulfillment_method', 'fulfillment_time', 'city', 'province', 'postal_code'], 'gold.daily_sales': ['date', 'orders_count', 'delivery_fees_count', 'delivery_fees_amount', 'priority_express_deli